In [27]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

In [28]:
train_df = pd.read_csv("assets/preprocessed_train.csv")
train_df

,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,0.352151,3,-0.216204,-0.395204,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,0.133379,8,4,208500
1,1,-5.199338,3,0.587964,0.079120,1,3,3,0,2,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.493553,-0.419741,8,4,181500
2,2,0.352151,3,-0.096753,0.630613,1,0,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.924252,0.133379,8,4,223500
3,3,0.723343,3,-0.526417,0.030279,1,0,3,0,0,...,2.449513,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,-5.199338,8,0,140000
4,4,0.352151,3,0.761353,1.348041,1,0,3,0,2,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,5.199338,0.133379,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,1455,0.352151,3,-0.344154,-0.585509,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.697507,-0.419741,8,4,175000
1090,1456,-5.199338,3,0.835558,1.107339,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,5.199338,8,4,210000
1091,1457,0.723343,3,-0.149853,-0.137177,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,5.199338,-0.493553,5.199338,8,4,266500
1092,1458,-5.199338,3,-0.096753,0.118777,1,3,3,0,4,...,1.295363,-5.199338,-5.199338,-5.199338,-5.199338,-0.860725,5.199338,8,4,142125


In [29]:
train_df.drop(columns=["Unnamed: 0"], inplace=True)
y = train_df["SalePrice"]
X = train_df.drop(columns=["SalePrice"])

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [31]:
y_train

6       307000
789     180000
1049    193000
837     117000
661     200500
         ...  
330      90350
466     118500
121     215000
1044    281213
860     174500
Name: SalePrice, Length: 875, dtype: int64

In [32]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
6,-5.199338,3,0.288750,0.241963,1,3,3,0,4,0,...,0.509202,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.697507,-0.419741,8,4
789,-5.199338,3,1.529558,0.667350,1,3,3,0,4,0,...,-5.199338,-5.199338,-5.199338,1.780005,-5.199338,-5.199338,-0.493553,0.133379,0,0
1049,0.352151,3,-0.344154,-0.736647,1,0,3,0,4,0,...,0.761353,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.860725,0.133379,8,4
837,-5.199338,3,1.053106,-1.228607,1,3,3,0,4,0,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.697507,-5.199338,8,4
661,0.352151,3,0.021329,-0.276998,1,3,3,0,4,0,...,0.782929,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.095492,5.199338,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,-0.261387,3,-1.449682,-1.425146,1,3,3,0,4,0,...,-5.199338,1.084524,-5.199338,-5.199338,-5.199338,-5.199338,-1.215372,-0.419741,8,4
466,-5.199338,3,-0.362850,-0.568672,1,3,3,0,4,0,...,-5.199338,1.097845,-5.199338,-5.199338,-5.199338,-5.199338,-0.860725,-0.419741,8,4
121,-5.199338,3,2.320818,2.291624,1,0,3,0,0,0,...,0.571638,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.493553,5.199338,8,4
1044,0.352151,3,0.969427,1.010382,1,0,3,0,4,0,...,0.205941,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,-0.419741,8,4


In [33]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse  


In [ ]:
study = optuna.create_study(direction="minimize")  # Minimize MSE
study.optimize(objective, n_trials=1000)  # Run for 50 trials

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

In [41]:
best_params = {
    "n_estimators": 1500,
    "learning_rate": 0.013036427795298889,
    "max_depth": 4,
    "subsample": 0.8089588318554315,
    "colsample_bytree": 0.7501883677918781,
    "gamma": 7.836327102470067,
    "reg_alpha": 0.191545255093672,
    "reg_lambda": 0.18442025407873489,
}
model = XGBRegressor(**best_params, random_state= 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")


RMSE: 22575.26669945009


# Using final preprocessed train

In [77]:
train_df = pd.read_csv("assets/final_preprocessed_train.csv")
train_df.drop(columns=["Unnamed: 0"], inplace=True)
y = train_df["SalePrice"]
X = train_df.drop(columns=["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [80]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse  

study = optuna.create_study(direction="minimize")  # Minimize MSE
study.optimize(objective, n_trials=500)  # Run for 50 trials

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

model = XGBRegressor(**study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

[I 2025-04-03 16:42:28,004] A new study created in memory with name: no-name-13e1e278-0a9b-4e3a-988c-d5f87bf439ff
[I 2025-04-03 16:42:28,561] Trial 0 finished with value: 26365.84523440021 and parameters: {'n_estimators': 500, 'learning_rate': 0.027452951579733785, 'max_depth': 6, 'subsample': 0.8368215561442194, 'colsample_bytree': 0.9923160098976753, 'gamma': 0.5240753111454888, 'reg_alpha': 0.09625945432681016, 'reg_lambda': 0.46911921691556735}. Best is trial 0 with value: 26365.84523440021.
[I 2025-04-03 16:42:28,835] Trial 1 finished with value: 27150.50737696976 and parameters: {'n_estimators': 300, 'learning_rate': 0.010340819970328685, 'max_depth': 4, 'subsample': 0.9256344688937931, 'colsample_bytree': 0.9003006452371873, 'gamma': 0.3147278100953055, 'reg_alpha': 0.7535756369594601, 'reg_lambda': 0.6383052028810318}. Best is trial 0 with value: 26365.84523440021.
[I 2025-04-03 16:42:30,971] Trial 2 finished with value: 26728.888438396825 and parameters: {'n_estimators': 400, 

Best hyperparameters: {'n_estimators': 600, 'learning_rate': 0.08583059363581344, 'max_depth': 4, 'subsample': 0.8146052079264698, 'colsample_bytree': 0.8811182032537543, 'gamma': 3.8693638455125585, 'reg_alpha': 0.9838376860974872, 'reg_lambda': 0.23815906137016088}
Best RMSE: 21201.73059939382
RMSE: 24180.267163642373


Removing columns based on correlation


In [81]:
corr_threshold = 0.1  
correlations = X_train.corrwith(y_train).abs()
to_drop = correlations[correlations < corr_threshold].index.tolist()
X_train_filtered = X_train.drop(columns=to_drop)
X_test_filtered = X_test.drop(columns=to_drop)

print(f"Removed {len(to_drop)} weakly correlated features: {to_drop}")
print(f"New shape of X_train: {X_train_filtered.shape}, X_test: {X_test_filtered.shape}")

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train_filtered, y_train)

    y_pred = model.predict(X_test_filtered)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse  

study = optuna.create_study(direction="minimize")  # Minimize MSE
study.optimize(objective, n_trials=500)  # Run for 50 trials

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)


model = XGBRegressor(**study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

c:\Users\Tanay\miniconda3\envs\ml\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Tanay\miniconda3\envs\ml\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
[I 2025-04-03 16:49:57,647] A new study created in memory with name: no-name-0be64317-f45e-4c7a-9902-83481e7c2cc4


Removed 24 weakly correlated features: ['MSSubClass', 'Street', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'Exterior1st', 'MasVnrType', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'PoolQC', 'MiscFeature', 'MiscVal', 'MoSold', 'SaleType']
New shape of X_train: (876, 54), X_test: (219, 54)


[I 2025-04-03 16:49:58,752] Trial 0 finished with value: 26277.79377072104 and parameters: {'n_estimators': 400, 'learning_rate': 0.11019875000562752, 'max_depth': 11, 'subsample': 0.745158146566745, 'colsample_bytree': 0.8850353314946472, 'gamma': 0.13870529930111286, 'reg_alpha': 0.28319595194717884, 'reg_lambda': 0.5123920354063928}. Best is trial 0 with value: 26277.79377072104.
[I 2025-04-03 16:49:59,374] Trial 1 finished with value: 28451.637210492358 and parameters: {'n_estimators': 500, 'learning_rate': 0.22581517569827236, 'max_depth': 11, 'subsample': 0.8444345874484152, 'colsample_bytree': 0.8663063499735218, 'gamma': 2.0894813613387155, 'reg_alpha': 0.7548990735290811, 'reg_lambda': 0.28556023123194385}. Best is trial 0 with value: 26277.79377072104.
[I 2025-04-03 16:50:01,453] Trial 2 finished with value: 27446.08977697754 and parameters: {'n_estimators': 1100, 'learning_rate': 0.05693756120816811, 'max_depth': 12, 'subsample': 0.5808547820073435, 'colsample_bytree': 0.990

Best hyperparameters: {'n_estimators': 1400, 'learning_rate': 0.05821921356314247, 'max_depth': 4, 'subsample': 0.84894961901351, 'colsample_bytree': 0.9721628913540785, 'gamma': 4.8619222788223775, 'reg_alpha': 0.09121451069001368, 'reg_lambda': 0.2835287472629321}
Best RMSE: 22177.315155092612
RMSE: 24593.466196517373


Using PCA to remove


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
col_to_remove_pca = categorical_features = [
    "MSZoning", "Street", "LotShape", "LandContour", "Utilities", "LotConfig",
    "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType",
    "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd",
    "ExterQual", "ExterCond", "Foundation", "BsmtQual", "BsmtCond",
    "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC",
    "CentralAir", "Electrical", "LowQualFinSF", "KitchenQual", "Functional",
    "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PavedDrive",
    "MiscVal", "SaleType", "SaleCondition"
]
X_train_filtered = X_train.drop(columns=col_to_remove_pca)
X_test_filtered = X_test.drop(columns=col_to_remove_pca)

print(f"Removed {len(to_drop)} weakly correlated features: {to_drop}")
print(f"New shape of X_train: {X_train_filtered.shape}, X_test: {X_test_filtered.shape}")
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train_filtered, y_train)

    y_pred = model.predict(X_test_filtered)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse  

study = optuna.create_study(direction="minimize")  # Minimize MSE
study.optimize(objective, n_trials=500)  # Run for 50 trials

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)


[I 2025-04-03 16:58:01,528] A new study created in memory with name: no-name-996cf4df-435e-4f0b-af30-b0bb75b19374


Removed 24 weakly correlated features: ['MSSubClass', 'Street', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'Exterior1st', 'MasVnrType', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'PoolQC', 'MiscFeature', 'MiscVal', 'MoSold', 'SaleType']
New shape of X_train: (876, 39), X_test: (219, 39)


[I 2025-04-03 16:58:02,968] Trial 0 finished with value: 33536.31101068029 and parameters: {'n_estimators': 1400, 'learning_rate': 0.0945286366636975, 'max_depth': 13, 'subsample': 0.8445674914379047, 'colsample_bytree': 0.9543352624329708, 'gamma': 6.577623015595238, 'reg_alpha': 0.00755687207368505, 'reg_lambda': 0.6250769417183913}. Best is trial 0 with value: 33536.31101068029.
[I 2025-04-03 16:58:05,192] Trial 1 finished with value: 35111.00543404192 and parameters: {'n_estimators': 1200, 'learning_rate': 0.05711445929106474, 'max_depth': 14, 'subsample': 0.5781120327143857, 'colsample_bytree': 0.6082930986845413, 'gamma': 6.237480519486739, 'reg_alpha': 0.039200707221577424, 'reg_lambda': 0.6591763406036667}. Best is trial 0 with value: 33536.31101068029.
[I 2025-04-03 16:58:05,945] Trial 2 finished with value: 36330.40012326937 and parameters: {'n_estimators': 1000, 'learning_rate': 0.2071662757542364, 'max_depth': 9, 'subsample': 0.6507259740327713, 'colsample_bytree': 0.791964

KeyboardInterrupt: 